<a href="https://colab.research.google.com/github/r73psv/univer/blob/master/ns_panova.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
#  Загрузка набора CIFAR10
# преобразовываем изображение в тензоры, затем нормализуем
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

100%|██████████| 170498071/170498071 [00:06<00:00, 28135233.47it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [3]:
#задаем устройство на котором будет тенироваться сеть - видеопроцессор, если нет то процессор
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [9]:
#создаем свёрточную НС (3 свёрточных слоя и 3 линейных)
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    # реализация метода forward в классе Net
    def forward(self, x):
        x = self.pool(F.logsigmoid(self.conv1(x)))
        x = self.pool(F.logsigmoid(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.logsigmoid(self.fc1(x))
        x = F.logsigmoid(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()
net.to(device)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [29]:
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

#criterion = nn.KLDivLoss()
criterion = nn.CrossEntropyLoss() # функция потерь
optimizer = optim.RMSprop(net.parameters(), lr=0.001, momentum=0.9) # функция оптимизации

for epoch in range(4):  # устанавливает количество циклов обучения
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):

        inputs, labels = data
        # обнуляем оптимизатор
        optimizer.zero_grad()
        # прямой проход
        outputs = net(inputs)
        out=inputs
        # Расчитываем функцию потерь
        loss = criterion(outputs, labels)
        #loss = criterion(inputs,out)
        # обратный проход
        loss.backward()
         # Шаг градиентного спуска (изменение параметров НС)
        optimizer.step()
        # печать статистики
        running_loss += loss.item()
        if i % 2000 == 1999:
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0
    print('Finished Training')

Finished Training
Finished Training
Finished Training
Finished Training


In [30]:
# сохраняем модель
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

In [31]:

dataiter = iter(testloader)
images, labels = next(dataiter)

In [32]:
#загрузка сохраненной модели
net = Net()
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [35]:
# Выходные данные представляют собой вероятности для 10 классов. Чем выше вероятность, тем больше сеть считает,
# что изображение относится к определенному классу.
outputs = net(images)
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join(f'{classes[predicted[j]]:5s}'
                              for j in range(4)))

Predicted:  ship  car   ship  ship 


In [36]:
correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        images, labels = data
        # вычисление выходных данных, прогоняя изображения по сети
        outputs = net(images)
        # класс с самой высокой вероятностью  выбираем в качестве прогноза
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 52 %


In [37]:
# подсчет прогнозов для каждого класса
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # собираем прогнозы длакаждого класа
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# печать accuracy для каждого класса
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

Accuracy for class: plane is 49.0 %
Accuracy for class: car   is 75.1 %
Accuracy for class: bird  is 24.0 %
Accuracy for class: cat   is 14.1 %
Accuracy for class: deer  is 38.0 %
Accuracy for class: dog   is 55.6 %
Accuracy for class: frog  is 76.6 %
Accuracy for class: horse is 62.9 %
Accuracy for class: ship  is 60.8 %
Accuracy for class: truck is 69.7 %
